# Threading

means using the multiples cores that modern CPUs have. Some things in Julia are automatically threaded, for instance, many linear algebra computations and the package manager. Also, many packages use are threaded. However, to get your own code threaded, there is some work to do (and some pitfalls to avoid).

# The Simplest Kind of Threading

in Julia is of this type:

```
using Base.Threads: @threads

Z = zeros(N)
@threads for i in 1:N
    local y
    y = SomeFunction(i,W)
    Z[i] = SomeOtherFunction(i,X,y,Z[i],but not Z[j])
end
U = YetAnotherFunction(Z)
```

In this case, `Z` is used for storing the results. Variables that are created in the loop (like `y`) should be declared `local` to avoid that they are shared across threads. Notice that is important to *not* let iteration `i` depend on results created by another iteration (`Z[j]`).

The `@threads` is a simple approach that works well when the iterations are similar (uniform). In contrast, if some iterations are much more costly or you want to use threads in a nested setting, the `@spawn/fetch/@sync` might be better.

Threading typically only pays off when the iterations involve heavy computations. Otherwise, the cost of setting of the threading might dominate. It is also important to avoid too many allocations (for instance, creating and deleting arrays) inside the threaded loop, or otherwise the memory allocation system ("gc") will impact the performance. 

# Activating Threading

Julia is (currently, as of version 1.10) started with a single thread. To change that default, set the environment variable `JULIA_NUM_THREADS=4` (it is often recommended to set it to the number of physical CPU cores).

In case you cannot change the environment variable, or simply want more fine-grained control, consider the following:

1. start julia from a command line as `julia --threads=4`

2. Set up jupyter/lab by first doing the following in the REPL
```
using IJulia
installkernel("Julia (4 threads)", env=Dict("JULIA_NUM_THREADS"=>"4"))
```
and then pick the right kernel when running the notebook.

3. In VScode, go to the Julia extension settings and search for `threads`. Then, change to `"julia.NumThreads": 4`

In [1]:
using Printf
using Base.Threads: @threads, @spawn         #load the threading functions

include("src/printmat.jl");

In [2]:
println("Number of threads: ", Threads.nthreads())   #check how many threads that are available

Number of threads: 4


# Comparing w/wo Threads 

We will calculate 

$\mu_{t}=\lambda \mu_{t-1} + (1-\lambda) x_{t-1}$, for $t=2...T$

on each column of a large matrix ($t$ refers to rows). 

The function `ExpMA(x,λ)` does the calculation for a vector (that is, a column in a bigger matrix). `ExpMA_1()` and `ExpMA_2()` loops over the columns without and with threading, respectively.

In [3]:
function ExpMA(x,λ)             #exponential MA, creates a vector
    T = length(x)
    μ  = zeros(T)
    for t in 2:T
        μ[t]  = λ*μ[t-1]  + (1-λ)*x[t-1]
    end
    return μ
end

ExpMA (generic function with 1 method)

In [4]:
function ExpMA_1(X,λ)          #wrap ExpMA(X,λ) in loop over columns of X
    (T,N) = (size(X,1),size(X,2))
    result = zeros(T,N)
    for i in 1:N
        result[:,i] = ExpMA(X[:,i],λ)
    end
    return result    
end

function ExpMA_2(X,λ)         #wrap but with threaded loop
    (T,N) = (size(X,1),size(X,2))
    result = zeros(T,N)
    @threads for i in 1:N      
        result[:,i] = ExpMA(X[:,i],λ)
    end
    return result    
end

ExpMA_2 (generic function with 1 method)

In [5]:
λ = 0.94
T = 100_000

N = 500
X = randn(T,N)

Y1 = ExpMA_1(X,λ)
Y2 = ExpMA_2(X,λ)

println("Test if the same results (Y1==Y2): ", Y1==Y2)

Test if the same results (Y1==Y2): true


## Avoiding (Memory) Allocations

is often a good idea. But, it becomes perhaps even more important in threaded applications.

The next cell defines a new function `ExpMAx!()` which writes the result to an existing matrix (the first function argument, `μ`) instead of creating a new vector every time. Then, the function `ExpMAx_2()` does a threaded loop over the columns in `X`. We verify that we get the same result as before.

In [6]:
function ExpMAx!(μ,x,λ,i)      #exponential MA, writes to column in an existing matrix μ 
    T = size(x,1)
    for t in 2:T
        μ[t,i]  = λ*μ[t-1,i]  + (1-λ)*x[t-1,i]
    end
    return μ
end

function ExpMAx_1(X,λ)             #no threading, but create
    (T,N) = (size(X,1),size(X,2))  #fewer intermediate vectors
    result = zeros(T,N)
    for i in 1:N      
        ExpMAx!(result,X,λ,i)
    end
    return result    
end


function ExpMAx_2(X,λ)             #wrap but with threaded loop and creating
    (T,N) = (size(X,1),size(X,2))  #fewer intermediate vectors
    result = zeros(T,N)
    @threads for i in 1:N      
        ExpMAx!(result,X,λ,i)
    end
    return result    
end

Y3 = ExpMAx_1(X,λ)
Y4 = ExpMAx_2(X,λ)
println("Test if the same results (Y1==Y3==Y4): ", Y1==Y3==Y4)

Test if the same results (Y1==Y3==Y4): true


## Speed Comparison

without thread, with threads (but creating intermediater vectors) and with threads but no intermediate vectors.

In [7]:
using BenchmarkTools           #package for benchmarking

printblue("standard loop:")                      
@btime ExpMA_1($X,λ)           #use $X to get more accurate timing

printblue("threaded loop:")
@btime ExpMA_2($X,λ)

printblue("standard loop, but without intermediate vectors:")
@btime ExpMAx_1($X,λ)

printblue("threaded loop, but without intermediate vectors:")
@btime ExpMAx_2($X,λ)

printmagenta("\n...reducing memory allocations and using threads can both speed up the code")

standard loop:
  717.284 ms (3003 allocations: 1.12 GiB)
threaded loop:
  191.757 ms (3027 allocations: 1.12 GiB)
standard loop, but without intermediate vectors:
  289.872 ms (3 allocations: 381.47 MiB)
threaded loop, but without intermediate vectors:
  118.300 ms (27 allocations: 381.47 MiB)

...reducing memory allocations and using threads can both speed up the code


# Things that Could Go Wrong with Threading (extra)

are often related to data races (different threads writing to the same memory location).

## Case 1: Several Threads Changing the Same Elements of an Array

When threads write to the same memory locations, anything can happen.

In [8]:
N = 10_000

x = [0]
@threads for i in 1:N
  x[1] = x[1] + 1            #all threads writing to x[1]
end

println("This ought to be $N, but it is ",x[])

This ought to be 10000, but it is 3851


## Case 2: Threads Writing to BitArrays

Code like in the next cell can also create unexpected results since the threads are trying write to the same BitArray (which has a packed format). Run a few times to see tha problem.

This is solved by instead using `Z = fill(false,N)` which is an array of Bools.

In [9]:
N = 10
Z = falses(N)
#Z = fill(false,N)              #ucomment to solve the problem
@threads for i in 1:N
    Z[i] = true
    sleep(0.5)                   #give the thread something to do
end

println("All $N values in Z should be `true`, but only ", sum(Z)," are. If needed, rerun to see.")

All 10 values in Z should be `true`, but only 9 are. If needed, rerun to see.


## Case 3: @threads and Variable Scope

Code like this
```
v = 1:2
@threads for i in 1:N
    v = something 
    x = SomeFunction(v)
end
```
can create unexpected results since the threads are sharing `v`. This is solved by declaring `v` inside the loop to be `local`.

In [10]:
using LinearAlgebra

function f2(N)
  v = falses(N+1)
  x = zeros(Int,N,N)
  @threads for i = 1:N
    #local v                   #uncomment to solve the problem
    v    = falses(N)
    v[i] = true
    x[v,i] .= i
  end
  return x
end

println("This should always be zero. Run a few times to check if that holds.\n")
M = 100
dev = zeros(M)
for i = 1:M
  dev[i] = maximum(abs,f2(i) - diagm(1:i))
end
println("All $M values should be 0, but only ", sum(dev.==0), " are.")

This should always be zero. Run a few times to check if that holds.

All 100 values should be 0, but only 8 are.
